## Base Line Model
with Local Binary Pattern, Mobile Net 3 small with 1 added layer, flat architecture.

### Imports

In [ ]:
import os
import pandas as pd
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.model_selection import train_test_split
import torch.nn as nn
import matplotlib.pyplot as plt
from PIL import Image
from skimage.feature import local_binary_pattern
from skimage import color
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### LBP Transform

In [ ]:
class LBPTransform:
    def __init__(self, radius=3, n_points=None, method='uniform'):
        self.radius = radius
        self.n_points = n_points if n_points else 8 * radius
        self.method = method

    def __call__(self, img):
        if isinstance(img, Image.Image):
            img = np.array(img)

        if len(img.shape) == 3 :
            gray = color.rgb2gray(img)
        else:
            gray = img

        gray = (gray * 255).astype(np.uint8)

        lbp = local_binary_pattern(gray, self.n_points, self.radius, self.method)

        lbp = (lbp - lbp.min()) / (lbp.max() - lbp.min() + 1e-7)

        lbp_3 = np.stack([lbp, lbp, lbp], axis=-1)

        return lbp_3

### Make LBP Images

In [ ]:
def make_lbp_images(input_folder, output_folder, lbp_transformer):
    os.makedirs(output_folder, exist_ok=True)
    img_files = [f for f in os.listdir(input_folder) if f.endswith('.png')]

    print(f"Processing {len(img_files)} images from {input_folder}...")
    for i, fname in enumerate(img_files):
        in_path = os.path.join(input_folder, fname)
        out_path = os.path.join(output_folder, fname)

        if os.path.exists(out_path):
            continue

        img = Image.open(in_path).convert('RGB')
        lbp_img = lbp_transformer(img)

        lbp_img_uint8 = (lbp_img * 255).astype(np.uint8)
        Image.fromarray(lbp_img_uint8).save(out_path)

    print("LBP preprocessing complete.")

### Dataset Class
Class for processing data and combining images with labels

In [ ]:
class PGCDataset(Dataset):
    def __init__(self, labels_df, img_folder, id_col='PGCname', label_col='T', transform=None):
        self.labels_df = labels_df.reset_index(drop=True)
        self.img_folder = img_folder
        self.id_col = id_col
        self.label_col = label_col
        self.transform = transform

        available_imgs = {f.replace('.png', '') for f in os.listdir(img_folder)
                            if f.endswith('.png')}
        self.labels_df = self.labels_df[self.labels_df[id_col].isin(available_imgs)].reset_index(drop=True)

        print(f"Dataset created with {len(self.labels_df)} imgs")

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        row = self.labels_df.iloc[idx]

        img_id = row[self.id_col]
        img_path = os.path.join(self.img_folder, f"{img_id}.png")
        img = Image.open(img_path).convert('RGB')

        label = torch.tensor(int(row[self.label_col]), dtype=torch.long)

        if self.transform:
            img = self.transform(img)

        return img, label, img_id

### Dataset creation

In [ ]:
path = '/content/drive/Othercomputers/My laptop/Thesis/'
img_folder = path + '/images'
lbp_img_folder = path + '/lbp_images'

id_col = 'PGCname'
label_col = 'T'

img_size = 224

labels_df = pd.read_csv(path + 'EFIGI_attributes.txt', sep=r'\s+', comment='#')
#labels_df[label_col] = labels_df[label_col].replace({-6:-4, -5:-4}) # E
labels_df[label_col] = labels_df[label_col].replace({-3:-2, -1:-2}) # S0
labels_df[label_col] = labels_df[label_col].replace({0:1, 2:1}) # Sa
labels_df[label_col] = labels_df[label_col].replace({3:4}) # Sb
labels_df[label_col] = labels_df[label_col].replace({5:6}) # Sc
labels_df[label_col] = labels_df[label_col].replace({8:7, 9:7}) # Sd
labels_df[label_col] = labels_df[label_col].replace({10:11}) # Irr

labels_df[label_col] = labels_df[label_col].replace({-6:0, -5:1, -4:2, -2:3, 1:4, 4:5, 11:8}) # Adjust to 0 - 8
#labels_df[label_col] = labels_df[label_col].replace({-4:0, -2:1, 1:2, 4:3, 6:4, 7:5, 11:6}) # Adjust to 0 - 6


train_df, test_df = train_test_split(labels_df, test_size=0.2, random_state=0, stratify=labels_df[label_col])
train_df, val_df = train_test_split(train_df, test_size=0.125, random_state=0, stratify=train_df[label_col])

# use stratify sampling in training - write to csv file - - tocsv.pandas


train_transform = transforms.Compose([
    transforms.RandomRotation(180),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

"""lbp_params = {'radius': 3, 'n_points': 24, 'method': 'uniform'}

lbp_transform = LBPTransform(**lbp_params)

make_lbp_images(img_folder, lbp_img_folder, lbp_transform)
"""


train_dataset = PGCDataset(
    labels_df=train_df,
    img_folder=img_folder,
    id_col=id_col,
    label_col=label_col,
    transform=train_transform
)
val_dataset = PGCDataset(
    labels_df=val_df,
    img_folder=img_folder,
    id_col=id_col,
    label_col=label_col,
    transform=test_transform
)
test_dataset = PGCDataset(
    labels_df=test_df,
    img_folder=img_folder,
    id_col=id_col,
    label_col=label_col,
    transform=test_transform
)

Dataset created with 3120 imgs
Dataset created with 446 imgs
Dataset created with 892 imgs


### Data loader
loads data in batches

In [ ]:
labels = train_df[label_col].values
classes= np.unique(labels)
class_weights = compute_class_weight('balanced', classes=classes, y=labels)

sample_weights = np.array([class_weights[np.where(classes == label)[0][0]] for label in labels])
sample_weights = torch.from_numpy(sample_weights).float()

sampler = torch.utils.data.WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)

train_loader = DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=0,
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=0,
)

test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=0
)

### Make model
using pretrained resnet18

In [ ]:
def resnet_model(num_classes, freeze_backbone=True):
    model = models.resnet18(weights='IMAGENET1K_V1')
    for param in model.parameters():
        param.requires_grad = not freeze_backbone
    in_features = model.fc.in_features
    model.fc = nn.Linear(in_features, num_classes)
    return model

## Train/test model methods

In [ ]:
def train_one_epoch(model, dataloader, criterion, optimizer, device, scaler=None):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for img, labels, ids in dataloader:
        img = img.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        optimizer.zero_grad()

        outputs = model(img)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        print(".", end="")
    print("")

    epoch_loss = running_loss / len(dataloader)
    epoch_acc = 100.0 * correct / total
    return epoch_loss, epoch_acc

def valid(model, dataloader, device, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for X, y, _ in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

            all_preds.extend(pred.argmax(1).cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    test_loss /= num_batches
    correct /= size
    return test_loss, correct * 100


def test(model, dataloader, device, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for X, y, _ in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

            all_preds.extend(pred.argmax(1).cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    test_loss /= num_batches
    correct /= size

    print(classification_report(all_labels, all_preds, digits=4))

    return test_loss, correct * 100


## Train model

In [ ]:
num_classes = labels_df[label_col].nunique()
model = resnet_model(num_classes=num_classes, freeze_backbone=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
torch.backends.cudnn.benchmark = True
print("Using device:", device)

class_weights = compute_class_weight('balanced', classes=np.unique(train_df[label_col]), y=train_df[label_col])
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
train_criterion = nn.CrossEntropyLoss(class_weights)

test_criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3)

best_acc = 0.0

epochs = 20
for epoch in range(epochs):
    train_loss, train_correct = train_one_epoch(model, train_loader, train_criterion, optimizer, device)
    val_loss, val_correct = valid(model, val_loader, device, test_criterion)

    print(f"Epoch {epoch+1}/{epochs}")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_correct:.2f}%")
    print(f"  Val   Loss: {val_loss:.4f} | Val   Acc: {val_correct:.2f}%")

    scheduler.step(val_loss)

    if val_correct > best_acc:
        best_acc = val_correct
        torch.save(model.state_dict(), path + 'baseline-50.pth')


Using device: cuda


KeyboardInterrupt: 

## Test model

In [ ]:
model = resnet_model(num_classes=num_classes, freeze_backbone=False)

model.load_state_dict(torch.load(path + 'baseline.pth'))
#resnet 18, stratified sampling, raw image data (pixel matric data)
model.to(device)
model.eval()

test_loss, correct = test(model, test_loader, device, test_criterion)

              precision    recall  f1-score   support

           0     0.4000    0.5000    0.4444         4
           1     0.7222    0.8667    0.7879        45
           2     0.3684    0.7778    0.5000         9
           3     0.8625    0.6449    0.7380       107
           4     0.6954    0.7778    0.7343       135
           5     0.7581    0.7121    0.7344       198
           6     0.6923    0.7200    0.7059       150
           7     0.8571    0.8287    0.8427       181
           8     0.8333    0.8730    0.8527        63

    accuracy                         0.7578       892
   macro avg     0.6877    0.7445    0.7045       892
weighted avg     0.7681    0.7578    0.7594       892

